In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn import metrics
import statsmodels.api as sm
from sklearn.linear_model import Lasso, Ridge


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
sns.set_context('paper')

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
train=pd.read_csv("/home/carlton/Downloads/abb_hackathon/train_v9rqX0R.csv")
test=pd.read_csv("/home/carlton/Downloads/abb_hackathon/test_AbJTz2l.csv")

## Data Analysis

In [4]:
train.Outlet_Size.value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

## Train  distribution of `Outlet_Size` across different `Outlet_Type` categories in the training dataset

In [5]:
crosstable = pd.crosstab(train['Outlet_Size'],train['Outlet_Type'])
crosstable

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Size,,,,
High,0,932,0,0
Medium,0,930,928,935
Small,528,1860,0,0


In [6]:
dic = {'Grocery Store':'Small'}
s = train.Outlet_Type.map(dic)

train.Outlet_Size= train.Outlet_Size.combine_first(s)
train.Outlet_Size.value_counts()

Small     2943
Medium    2793
High       932
Name: Outlet_Size, dtype: int64

# Test Train distribution of Outlet_Size across different Outlet_Type categories in the training dataset

In [7]:
crosstable = pd.crosstab(test['Outlet_Size'],test['Outlet_Type'])
crosstable

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Size,,,,
High,0,621,0,0
Medium,0,620,618,624
Small,352,1240,0,0


In [8]:
dic = {'Grocery Store':'Small'}
s = test.Outlet_Type.map(dic)

test.Outlet_Size= test.Outlet_Size.combine_first(s)
test.Outlet_Size.value_counts()

Small     1962
Medium    1862
High       621
Name: Outlet_Size, dtype: int64

In [9]:
train.isnull().sum(axis=0)

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1855
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [10]:
test.isnull().sum(axis=0)

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1236
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

### Distribution of Outlet Size across Outlet Location Type


In [11]:
train.Outlet_Size.value_counts()

Small     2943
Medium    2793
High       932
Name: Outlet_Size, dtype: int64

In [12]:
#checking for location type
crosstable = pd.crosstab(train.Outlet_Size,train.Outlet_Location_Type)
crosstable

Outlet_Location_Type,Tier 1,Tier 2,Tier 3
Outlet_Size,,,
High,0,0,932
Medium,930,0,1863
Small,1458,930,555


### Outlet Size Imputation Logic

Mapped all `Outlet_Location_Type == "Tier 2"` to `Outlet_Size = "Small"` for missing values, then displayed the updated counts of `Outlet_Size`.


In [13]:
dic = {"Tier 2":"Small"}
s = train.Outlet_Location_Type.map(dic)
train.Outlet_Size = train.Outlet_Size.combine_first(s)
train.Outlet_Size.value_counts()

Small     4798
Medium    2793
High       932
Name: Outlet_Size, dtype: int64

In [14]:
test.Outlet_Size.value_counts()

Small     1962
Medium    1862
High       621
Name: Outlet_Size, dtype: int64

In [15]:
#checking for location type
crosstable = pd.crosstab(test.Outlet_Size,test.Outlet_Location_Type)
crosstable

Outlet_Location_Type,Tier 1,Tier 2,Tier 3
Outlet_Size,,,
High,0,0,621
Medium,620,0,1242
Small,972,620,370


doubt

In [16]:
dic = {"Tier 2":"Small"}
s = test.Outlet_Location_Type.map(dic)
test.Outlet_Size = test.Outlet_Size.combine_first(s)
test.Outlet_Size.value_counts()

Small     3198
Medium    1862
High       621
Name: Outlet_Size, dtype: int64

### Imputing Outlet Size in Test Data Using Outlet Type

Filled missing `Outlet_Size` values in the test set using a predefined mapping from `Outlet_Type` to `Outlet_Size` based on distribution insights. Finally, printed the number of remaining missing values after imputation.


In [17]:
# Mapping based on your outlet_type vs size distribution
size_map = {
    'Grocery Store': 'Small',
    'Supermarket Type1': 'Small',
    'Supermarket Type2': 'Medium',
    'Supermarket Type3': 'Medium'
}

# Fill Outlet_Size using this mapping
test['Outlet_Size'] = test['Outlet_Size'].combine_first(
    test['Outlet_Type'].map(size_map)
)

# Optional: check missing after fill
print("Remaining NaNs:", test['Outlet_Size'].isna().sum())


Remaining NaNs: 0


In [18]:
train.isnull().sum(axis=0)

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [19]:
test.isnull().sum(axis=0)

Item_Identifier                0
Item_Weight                  976
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                    0
Outlet_Location_Type           0
Outlet_Type                    0
dtype: int64

### Imputing Missing Item Weight in Training Data

Missing values in `Item_Weight` are filled using the mean weight per `Item_Identifier`, ensuring consistent and specific imputation. The remaining missing values are then summarized.


In [20]:
train['Item_Weight']=train['Item_Weight'].fillna(train.groupby('Item_Identifier')['Item_Weight'].transform('mean'))
train.isnull().sum()

Item_Identifier              0
Item_Weight                  4
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [21]:
train[train.Item_Weight.isnull()]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


In [22]:
# List of item types 
item_type_list = train.Item_Type.unique().tolist()
item_type_list

['Dairy',
 'Soft Drinks',
 'Meat',
 'Fruits and Vegetables',
 'Household',
 'Baking Goods',
 'Snack Foods',
 'Frozen Foods',
 'Breakfast',
 'Health and Hygiene',
 'Hard Drinks',
 'Canned',
 'Breads',
 'Starchy Foods',
 'Others',
 'Seafood']

In [23]:
# grouping based on item type and calculating mean of item weight
Item_Type_Means = train.groupby('Item_Type')['Item_Weight'].mean() 

In [24]:
Item_Type_Means

Item_Type
Baking Goods             12.285317
Breads                   11.297689
Breakfast                12.779727
Canned                   12.403320
Dairy                    13.379905
Frozen Foods             12.782404
Fruits and Vegetables    13.236713
Hard Drinks              11.456238
Health and Hygiene       13.052327
Household                13.524780
Meat                     12.771212
Others                   13.979438
Seafood                  12.521953
Snack Foods              13.031230
Soft Drinks              11.879775
Starchy Foods            13.841385
Name: Item_Weight, dtype: float64

### Imputing Missing Item Weight Using Item Type Means

For each `Item_Type`, missing `Item_Weight` values are filled with the corresponding mean from `Item_Type_Means`. This ensures more context-aware imputation rather than using a global average.


In [29]:
# Mapiing Item weight to item type mean
for i in item_type_list:
    dic = {i:Item_Type_Means[i]}
    s = train.Item_Type.map(dic)
    train.Item_Weight = train.Item_Weight.combine_first(s)
    
Item_Type_Means = train.groupby('Item_Type')['Item_Weight'].mean() 

train.isnull().sum()


Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

### Imputing Missing Item Weight in Test Data

Filled missing `Item_Weight` values in the test set using the mean weight of each `Item_Identifier`. This preserves item-specific consistency. Displayed the count of remaining missing values after imputation.


In [30]:
test['Item_Weight']=test['Item_Weight'].fillna(test.groupby('Item_Identifier')['Item_Weight'].transform('mean'))
test.isnull().sum()

Item_Identifier               0
Item_Weight                  20
Item_Fat_Content              0
Item_Visibility               0
Item_Type                     0
Item_MRP                      0
Outlet_Identifier             0
Outlet_Establishment_Year     0
Outlet_Size                   0
Outlet_Location_Type          0
Outlet_Type                   0
dtype: int64

In [31]:
# List of item types 
item_type_list = test.Item_Type.unique().tolist()
item_type_list

['Snack Foods',
 'Dairy',
 'Others',
 'Fruits and Vegetables',
 'Baking Goods',
 'Health and Hygiene',
 'Breads',
 'Hard Drinks',
 'Seafood',
 'Soft Drinks',
 'Household',
 'Frozen Foods',
 'Meat',
 'Canned',
 'Starchy Foods',
 'Breakfast']

In [32]:
# grouping based on item type and calculating mean of item weight
Item_Type_Means = test.groupby('Item_Type')['Item_Weight'].mean() 

In [33]:
# Mapiing Item weight to item type mean
for i in item_type_list:
    dic = {i:Item_Type_Means[i]}
    s = test.Item_Type.map(dic)
    test.Item_Weight = test.Item_Weight.combine_first(s)
    
Item_Type_Means = test.groupby('Item_Type')['Item_Weight'].mean() 

test.isnull().sum()


Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [34]:
test[test.Item_Weight.isnull()]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type


In [35]:
train.Item_Visibility.value_counts().head() 

0.000000    526
0.076975      3
0.041283      2
0.085622      2
0.187841      2
Name: Item_Visibility, dtype: int64

In [36]:
import numpy as np
# Replacing 0's with NaN
train.Item_Visibility.replace(to_replace=0.000000,value=np.NaN,inplace=True)
# Now fill by mean of visbility based on item identifiers
train.Item_Visibility = train.Item_Visibility.fillna(train.groupby('Item_Identifier')['Item_Visibility'].transform('mean'))


In [37]:
train.Item_Visibility.value_counts().head()

0.121880    4
0.082138    3
0.016164    3
0.029511    3
0.081428    3
Name: Item_Visibility, dtype: int64

In [38]:
test.Item_Visibility.value_counts().head() 

0.000000    353
0.107493      2
0.159518      2
0.026711      2
0.056306      2
Name: Item_Visibility, dtype: int64

In [39]:
import numpy as np
# Replacing 0's with NaN
test.Item_Visibility.replace(to_replace=0.000000,value=np.NaN,inplace=True)
# Now fill by mean of visbility based on item identifiers
test.Item_Visibility = test.Item_Visibility.fillna(test.groupby('Item_Identifier')['Item_Visibility'].transform('mean'))


In [40]:
test.Item_Visibility.value_counts().head()

0.012002    3
0.065787    3
0.127416    3
0.072816    3
0.017073    2
Name: Item_Visibility, dtype: int64

In [41]:
test['Item_Visibility'].fillna(test['Item_Visibility'].mean(), inplace=True)


In [42]:
# test['Item_Visibility'] = test['Item_Visibility'].fillna(
#     test.groupby('Item_Identifier')['Item_Visibility'].transform('mean')
# )


In [43]:

test.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [44]:
train.Item_Fat_Content.value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [45]:
train.Item_Fat_Content.replace(to_replace=["LF","low fat"],value="Low Fat",inplace=True)
train.Item_Fat_Content.replace(to_replace="reg",value="Regular",inplace=True)

train.Item_Fat_Content.value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [46]:
test.Item_Fat_Content.value_counts()

Low Fat    3396
Regular    1935
LF          206
reg          78
low fat      66
Name: Item_Fat_Content, dtype: int64

In [47]:
test.Item_Fat_Content.replace(to_replace=["LF","low fat"],value="Low Fat",inplace=True)
test.Item_Fat_Content.replace(to_replace="reg",value="Regular",inplace=True)

test.Item_Fat_Content.value_counts()

Low Fat    3668
Regular    2013
Name: Item_Fat_Content, dtype: int64

In [48]:
train['Outlet_Year'] = (2013 - train.Outlet_Establishment_Year)

In [49]:
test['Outlet_Year'] = (2013 - test.Outlet_Establishment_Year)

In [50]:
var_cat = train.select_dtypes(include=[object])
var_cat.head()

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,Low Fat,Dairy,OUT049,Medium,Tier 1,Supermarket Type1
1,DRC01,Regular,Soft Drinks,OUT018,Medium,Tier 3,Supermarket Type2
2,FDN15,Low Fat,Meat,OUT049,Medium,Tier 1,Supermarket Type1
3,FDX07,Regular,Fruits and Vegetables,OUT010,Small,Tier 3,Grocery Store
4,NCD19,Low Fat,Household,OUT013,High,Tier 3,Supermarket Type1


In [51]:
#Convert categorical into numerical 
var_cat = var_cat.columns.tolist()
var_cat = ['Item_Fat_Content',
 'Item_Type',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

var_cat

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [52]:
train['Item_Type_New'] = train.Item_Identifier
train.Item_Type_New.head(10)

0    FDA15
1    DRC01
2    FDN15
3    FDX07
4    NCD19
5    FDP36
6    FDO10
7    FDP10
8    FDH17
9    FDU28
Name: Item_Type_New, dtype: object

In [53]:
train.Item_Type_New.replace(to_replace="^FD*.*",value="Food",regex=True,inplace=True)
train.Item_Type_New.replace(to_replace="^DR*.*",value="Drinks",regex=True,inplace=True)
train.Item_Type_New.replace(to_replace="^NC*.*",value="Non-Consumable",regex=True,inplace=True)

train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Year,Item_Type_New
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,14,Food
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,4,Drinks
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,14,Food
3,FDX07,19.20,Regular,0.022911,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,15,Food
4,NCD19,8.93,Low Fat,0.016164,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,26,Non-Consumable


In [54]:
test

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Outlet_Year
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,14
1,FDW14,8.300,Regular,0.038428,Dairy,87.3198,OUT017,2007,Small,Tier 2,Supermarket Type1,6
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,Small,Tier 3,Grocery Store,15
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,Small,Tier 2,Supermarket Type1,6
4,FDY38,13.600,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,28
5,FDH56,9.800,Regular,0.063817,Fruits and Vegetables,117.1492,OUT046,1997,Small,Tier 1,Supermarket Type1,16
6,FDL48,19.350,Regular,0.082602,Baking Goods,50.1034,OUT018,2009,Medium,Tier 3,Supermarket Type2,4
7,FDC48,9.195,Low Fat,0.015782,Baking Goods,81.0592,OUT027,1985,Medium,Tier 3,Supermarket Type3,28
8,FDN33,6.305,Regular,0.123365,Snack Foods,95.7436,OUT045,2002,Small,Tier 2,Supermarket Type1,11
9,FDA36,5.985,Low Fat,0.005698,Baking Goods,186.8924,OUT017,2007,Small,Tier 2,Supermarket Type1,6


In [55]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_outlet = LabelEncoder()
le_item = LabelEncoder()

In [56]:
train['Outlet'] = le_outlet.fit_transform(train.Outlet_Identifier)
train['Item'] = le_item.fit_transform(train.Item_Type_New)
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Year,Item_Type_New,Outlet,Item
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,14,Food,9,1
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,4,Drinks,3,0
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,14,Food,9,1
3,FDX07,19.20,Regular,0.022911,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,15,Food,0,1
4,NCD19,8.93,Low Fat,0.016164,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,26,Non-Consumable,1,2


In [57]:
from sklearn.preprocessing import LabelEncoder

le_fat = LabelEncoder()
train['Item_Fat_Content'] = le_fat.fit_transform(train['Item_Fat_Content'].astype(str))

le_type = LabelEncoder()
train['Item_Type'] = le_type.fit_transform(train['Item_Type'].astype(str))

le_size = LabelEncoder()
train['Outlet_Size'] = le_size.fit_transform(train['Outlet_Size'].astype(str))

le_location = LabelEncoder()
train['Outlet_Location_Type'] = le_location.fit_transform(train['Outlet_Location_Type'].astype(str))

le_otype = LabelEncoder()
train['Outlet_Type'] = le_otype.fit_transform(train['Outlet_Type'].astype(str))


In [58]:
test['Item_Type_New'] = test.Item_Identifier
test.Item_Type_New.head(10)

0    FDW58
1    FDW14
2    NCN55
3    FDQ58
4    FDY38
5    FDH56
6    FDL48
7    FDC48
8    FDN33
9    FDA36
Name: Item_Type_New, dtype: object

In [59]:
test.Item_Type_New.replace(to_replace="^FD*.*",value="Food",regex=True,inplace=True)
test.Item_Type_New.replace(to_replace="^DR*.*",value="Drinks",regex=True,inplace=True)
test.Item_Type_New.replace(to_replace="^NC*.*",value="Non-Consumable",regex=True,inplace=True)

test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Outlet_Year,Item_Type_New
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,14,Food
1,FDW14,8.300,Regular,0.038428,Dairy,87.3198,OUT017,2007,Small,Tier 2,Supermarket Type1,6,Food
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,Small,Tier 3,Grocery Store,15,Non-Consumable
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,Small,Tier 2,Supermarket Type1,6,Food
4,FDY38,13.600,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,28,Food


In [60]:
def safe_label_transform(le, values, default=-1):
    known = set(le.classes_)
    return [le.transform([v])[0] if v in known else default for v in values]

test['Outlet'] = safe_label_transform(le_outlet, test['Outlet_Identifier'])
test['Item'] = safe_label_transform(le_item, test['Item_Type_New'])


In [61]:
# Safe transform for unseen labels
def safe_label_transform(le, values, default=-1):
    known = set(le.classes_)
    return [le.transform([v])[0] if v in known else default for v in values]

# Apply train-fitted encoders to test data
test['Item_Fat_Content'] = safe_label_transform(le_fat, test['Item_Fat_Content'].astype(str))
test['Item_Type'] = safe_label_transform(le_type, test['Item_Type'].astype(str))
test['Outlet_Size'] = safe_label_transform(le_size, test['Outlet_Size'].astype(str))
test['Outlet_Location_Type'] = safe_label_transform(le_location, test['Outlet_Location_Type'].astype(str))
test['Outlet_Type'] = safe_label_transform(le_otype, test['Outlet_Type'].astype(str))


In [62]:
#Visualizing Correlation
corrmat = train.corr()
corrmat
# corrmat.to_csv("original_.csv")

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Year,Outlet,Item
Item_Weight,1.000000,-0.026797,-0.021226,0.035710,0.025967,-0.013417,-0.012445,0.002973,0.000592,0.013198,0.013417,-0.007586,0.071585
Item_Fat_Content,-0.026797,1.000000,0.052620,-0.139434,0.006063,0.003151,-0.001262,-0.001598,0.002199,0.018719,-0.003151,0.000764,-0.166111
Item_Visibility,-0.021226,0.052620,1.000000,-0.042400,-0.005951,-0.075238,0.090322,-0.030449,-0.170752,-0.126026,0.075238,-0.096282,-0.050520
Item_Type,0.035710,-0.139434,-0.042400,1.000000,0.032651,0.004970,-0.000218,0.003084,0.003053,0.017048,-0.004970,0.001656,0.007456
Item_MRP,0.025967,0.006063,-0.005951,0.032651,1.000000,0.005020,0.000872,0.000232,-0.001975,0.567574,-0.005020,0.003319,0.032517
Outlet_Establishment_Year,-0.013417,0.003151,-0.075238,0.004970,0.005020,1.000000,0.425534,-0.089216,-0.122304,-0.049135,-1.000000,0.079035,-0.008551
Outlet_Size,-0.012445,-0.001262,0.090322,-0.000218,0.000872,0.425534,1.000000,-0.480075,-0.401373,-0.162753,-0.425534,0.260272,-0.001276
Outlet_Location_Type,0.002973,-0.001598,-0.030449,0.003084,0.000232,-0.089216,-0.480075,1.000000,0.467219,0.089367,0.089216,-0.716176,0.007661
Outlet_Type,0.000592,0.002199,-0.170752,0.003053,-0.001975,-0.122304,-0.401373,0.467219,1.000000,0.401522,0.122304,0.099873,0.001136
Item_Outlet_Sales,0.013198,0.018719,-0.126026,0.017048,0.567574,-0.049135,-0.162753,0.089367,0.401522,1.000000,0.049135,0.162325,0.011236


# Predictive Modelling

In [63]:
seed = 240
np.random.seed(seed)

In [64]:
predictors=['Item_MRP','Outlet_Size','Outlet_Location_Type','Outlet_Type','Outlet_Year']

In [65]:
extra_features = [col for col in test.columns if col not in predictors]
print("🟨 Columns in train but not in predictors:", extra_features)

🟨 Columns in train but not in predictors: ['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Item_Type_New', 'Outlet', 'Item']


In [66]:
train['Outlet_NM'] = train['Outlet_Identifier'].str[-2:].astype(int)
test['Outlet_NM'] = test['Outlet_Identifier'].str[-2:].astype(int)

In [67]:
def add_item_category_onehot(train, test):
    # Step 1: Extract category
    train['Item_Category'] = train['Item_Identifier'].str[:2]
    test['Item_Category'] = test['Item_Identifier'].str[:2]

    # Step 2: One-hot encode
    train_dummies = pd.get_dummies(train['Item_Category'], prefix='ItemCat')
    test_dummies = pd.get_dummies(test['Item_Category'], prefix='ItemCat')

    # Step 3: Align columns
    train_dummies, test_dummies = train_dummies.align(test_dummies, join='outer', axis=1, fill_value=0)

    # Step 4: Append to original DataFrames
    train = pd.concat([train, train_dummies], axis=1)
    test = pd.concat([test, test_dummies], axis=1)

    # Optional: Drop the intermediate column
#     train.drop(columns='Item_Category', inplace=True)
#     test.drop(columns='Item_Category', inplace=True)

    return train, test

In [68]:
def encode_item_category(train, test):
    mapping = {'FD': 1, 'NC': 2, 'DR': 3}
    train['Item_Category_Code'] = train['Item_Identifier'].str[:2].map(mapping)
    test['Item_Category_Code'] = test['Item_Identifier'].str[:2].map(mapping)
    return train, test

In [69]:
train, test = encode_item_category(train, test)

In [70]:


train, test = add_item_category_onehot(train, test)

In [71]:
train[extra_features]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Outlet_Identifier,Outlet_Establishment_Year,Item_Type_New,Outlet,Item
0,FDA15,9.300,0,0.016047,4,OUT049,1999,Food,9,1
1,DRC01,5.920,1,0.019278,14,OUT018,2009,Drinks,3,0
2,FDN15,17.500,0,0.016760,10,OUT049,1999,Food,9,1
3,FDX07,19.200,1,0.022911,6,OUT010,1998,Food,0,1
4,NCD19,8.930,0,0.016164,9,OUT013,1987,Non-Consumable,1,2
5,FDP36,10.395,1,0.091392,0,OUT018,2009,Food,3,1
6,FDO10,13.650,1,0.012741,13,OUT013,1987,Food,1,1
7,FDP10,19.000,0,0.127470,13,OUT027,1985,Food,5,1
8,FDH17,16.200,1,0.016687,5,OUT045,2002,Food,7,1
9,FDU28,19.200,1,0.094450,5,OUT017,2007,Food,2,1


In [72]:
X = train[predictors]
y = train.Item_Outlet_Sales

In [73]:
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 42)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
     # must be a column in X
)


In [75]:
X_train.shape

(6392, 5)